In [10]:
import tensorflow as tf
import numpy as np

In [4]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [5]:
#tf.matmul即矩阵相乘，满足前面的列数等于后面的行数即可
a = tf.constant([1, 2, 3, 4, 5, 6], shape=[2, 3]) # [[1. 2. 3.],[4. 5. 6.]]
b = tf.constant([7, 8, 9, 10, 11, 12], shape=[3, 2]) # [[7. 8.],[9. 10.],[11. 12.]]
c = tf.matmul(a, b) # [[58 64],[139 154]]
print(sess.run(c))

[[ 58  64]
 [139 154]]


In [8]:
#tf.multiply即矩阵对应元素相乘，要求两个矩阵必须同等大小
x = tf.constant([[1.0,2.0,3.0],[1.0,2.0,3.0],[1.0,2.0,3.0]])  
y = tf.constant([[1.,0,0],[0,1,0],[0,0,1]])
z = tf.multiply(x,y) 
print(sess.run(z))

[[1. 0. 0.]
 [0. 2. 0.]
 [0. 0. 3.]]


In [14]:
#特殊的，应用于词向量的余弦相似度上
#word2vec中求若干个词向量与总的词向量中相关的
data = np.random.randint(0,10,(1000,120)) #这里表示共1000个词语或字，每个词语或字用120维的一维特征向量表示
data = data.astype(np.float32)
norm = tf.sqrt(tf.reduce_sum(tf.square(data), 1, keep_dims=True)) # 对每行元素进行平方和后开方，主要为了单位化每一行向量
normalized_embeddings = data / norm #相当于对1000个向量，每个向量都做了单位化，主要是为了后面求余弦时，可以直接等于向量点积
print(sess.run(normalized_embeddings))
valid_examples = np.random.choice(100, 10, replace=False) #即从[0,100)里随机不重复挑选10个数
valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset) #从总表中挑选出对应数字的十行
#这里表示我随机从总表中随机挑选出10个词语

[[0.11610363 0.08293117 0.         ... 0.13268988 0.08293117 0.09951741]
 [0.01720307 0.08601534 0.03440613 ... 0.12042147 0.0516092  0.        ]
 [0.05259921 0.15779763 0.08766535 ... 0.10519842 0.08766535 0.05259921]
 ...
 [0.10163654 0.03387885 0.15245481 ... 0.         0.01693942 0.11857597]
 [0.05084015 0.03389343 0.1016803  ... 0.11862702 0.15252045 0.        ]
 [0.01808049 0.14464395 0.         ... 0.03616099 0.16272445 0.14464395]]


In [ ]:
similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)
#这里valid_embeddings是从总表中随机挑取的十个字对应的字或者词向量，shape= 10 * 120
#这里normalized_embeddings即总表，代表所有词或字的向量表，shape=1000 * 120

#tf.matmul，我们知道是矩阵相乘，transpose_a和transpose_b默认都为False，第一个代表a是否要转置，第二个代表b是否要转置
#因为是矩阵相乘，故而这里b需要转置，从而使得前一个矩阵列数等于后一个矩阵行数

#两个字或者两个词是否相似，我们用这两个字或词的向量进行求余弦可得，即余弦=(向量a与向量b的点积)/(向量a和向量b的模长之积)
#两个向量点积即对应元素相乘相加，若是单位向量的话，则分母为1，从而余弦=两个向量对应位置元素相乘相加

#这里我们有十个字/词的向量，还有整个字典的字/词向量,而十个字/词是从整个字典中随机取来的，整个词典中每个字/词都是单位向量
#valid_embeddings 与 normalized_embeddings的转置 两矩阵相乘，即 10*120 与 120*1000 相乘，得到10*1000的矩阵
#理解一下，similarity矩阵中每一个元素即是valid_embeddings矩阵中的一个字与normalized_embeddings矩阵中的一个字做的点积
#故而similarity矩阵中每一个元素都相当于是两个字/词的点积结果，即用10个字/词与1000个字/词进行求余弦相似度了。
#值越大，也就是越接近于1的，代表这两个矩阵中有相似的词。
#比如similarity矩阵中第i行第j列元素值为1，其结果是valid_embeddings中第i行与normalized_embeddings第j列相乘的结果
#即代表valid_embeddings的第i个字/词与normalized_embeddings的第j个字/词相似。

#故而要想知道在总表中，与valid_embeddings中第i个字相似的，则去similarity矩阵中的第i行寻找。
#值越大的，代表越相似，并且，该词处于similarity中第j列，则代表是总表normalized_embeddings中第j个字与其相似的